In [1]:
import numpy as np
import datetime
import pandas as pd
import os
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

### ACF, PACF EDA ###
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA


# LSTM generator
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
# minmax scale 돌린 dataset
store_group_minmax_scaled = pd.read_csv(os.getcwd()+'\\store_group_minmax.csv')
distinct_store_names = store_group_minmax_scaled.columns[1:11]

In [28]:
def LSTM_model(x):
    
    global store_group_minmax_scaled
    global distinct_store_names
    
    # 어느 가게 타겟할건지
    target_store = distinct_store_names[x]
    
    # scaled data 설정
    data_scaled = store_group_minmax_scaled[target_store].ravel().reshape(-1,1)
    
    # train, test split
    train_data, test_data = data_scaled[:len(data_scaled) - 56], data_scaled[len(data_scaled) - 56 : ]
    
    # 시계열 Generator
    
    n_input = 28
    n_features= 1
    
    train_generator = TimeseriesGenerator(train_data, train_data, length=n_input, batch_size=1)
    test_generator = TimeseriesGenerator(test_data, test_data, length=n_input, batch_size=1)
    
    # LSTM model
    
    model = Sequential()
    model.add(LSTM(4, activation = 'tanh', input_shape=(n_input, 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    history = model.fit_generator(train_generator, epochs=32).history
    yhat = model.predict(test_generator)
    
    validation_data = data_scaled[len(data_scaled) - 28 : ]
    
    # plot 그리기 
    
    plt.plot(range(len(yhat.ravel())), predicted.ravel(), c='r', label = "predicted data")
    plt.plot(range(len(yhat.ravel())), validation.ravel(), label = 'real data')
    plt.legend()
    
    # MSE
    
    return mean_squared_error(validation, predicted)